In [1]:
import time
import json
from numpy import array, mean
import pandas as pd
import requests

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.sql.functions import year, to_date, date_format
from pyspark.sql.functions import from_json, col, udf, schema_of_json, size, explode, collect_set, map_keys
from pyspark.sql.functions import isnan, when, count, col,lower, split, trim, concat_ws
from pyspark.sql.types import BooleanType, MapType, StructType, StructField, StringType

from pyspark.sql.functions import col, radians, sin, cos, atan2, sqrt
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import math

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
spark = SparkSession.builder \
.appName('Group11') \
.config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
.getOrCreate()

# #change configuration settings on Spark 
# conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

# spark.sparkContext.getConf().getAll()
#set log level
spark.sparkContext.setLogLevel("ERROR")

In [3]:
#review configuration settings
spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

In [4]:
# Read dataset
# base_dir="gs://andrew-bigdata/yelp_dataset/"
# base_dir="/home/jovyan/work/dataset_weather.csv"
# base_dir2="/home/jovyan/work/Combined_Flights_2022.csv"

weather_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/dataset_weather.csv")
flight_2018_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/Combined_Flights_2018.csv")
flight_2019_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/Combined_Flights_2019.csv")
flight_2020_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/Combined_Flights_2020.csv")
flight_2021_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/Combined_Flights_2021.csv")

airport_df = spark.read.option("header", "true").csv("gs://msca-bdp-student-gcs/Group11/usa-airports.csv")

In [5]:
weather_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- EVAP: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Elevation: string (nullable = true)



In [8]:
flight_2021_df.printSchema()

root
 |-- FlightDate: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelayMinutes: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelayMinutes: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT

In [9]:
airport_df.printSchema()

root
 |-- iata: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [10]:
weather_df.show(2, vertical = True)

-RECORD 0----------------
 ID        | USS0012M13S 
 DATE      | 3/26/1997   
 TMAX      | null        
 TMIN      | null        
 EVAP      | null        
 PRCP      | 0           
 Latitude  | 37.66       
 Longitude | -112.74     
 Elevation | 2920        
-RECORD 1----------------
 ID        | US1OKCV0021 
 DATE      | 4/18/2007   
 TMAX      | null        
 TMIN      | null        
 EVAP      | null        
 PRCP      | 241         
 Latitude  | 35.1715     
 Longitude | -97.4262    
 Elevation | 355.1       
only showing top 2 rows



In [12]:
flight_2021_df.show(1, vertical = True)

24/11/04 02:30:57 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------------------
 FlightDate                              | 2021-03-03           
 Airline                                 | SkyWest Airlines ... 
 Origin                                  | SGU                  
 Dest                                    | PHX                  
 Cancelled                               | False                
 Diverted                                | False                
 CRSDepTime                              | 724                  
 DepTime                                 | 714.0                
 DepDelayMinutes                         | 0.0                  
 DepDelay                                | -10.0                
 ArrTime                                 | 818.0                
 ArrDelayMinutes                         | 0.0                  
 AirTime                                 | 49.0                 
 CRSElapsedTime                          | 79.0                 
 ActualElapsedTime       

In [13]:
airport_df.show(2, vertical =  True)

-RECORD 0-------------------------
 iata      | 00M                  
 name      | Thigpen              
 city      | Bay Springs          
 state     | MS                   
 country   | USA                  
 latitude  | 31.95376472          
 longitude | -89.23450472         
-RECORD 1-------------------------
 iata      | 00R                  
 name      | Livingston Municipal 
 city      | Livingston           
 state     | TX                   
 country   | USA                  
 latitude  | 30.68586111          
 longitude | -95.01792778         
only showing top 2 rows



In [14]:
#display number of records by partition
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

In [15]:
#number of partitions
weather_df.rdd.getNumPartitions()

63

In [17]:
flight_2021_df.rdd.getNumPartitions()

17

In [18]:
displaypartitions(weather_df)

Partitions: 63


+-----------+-------+
|partitionId|  count|
+-----------+-------+
|         62| 860372|
|         12|2498648|
|         20|2498797|
|         26|2498834|
|         52|2498906|
|         37|2498909|
|         41|2499026|
|         16|2499035|
|         22|2499044|
|         30|2499089|
|         45|2499137|
|          7|2499160|
|          4|2499178|
|         34|2499194|
|         60|2499221|
|         49|2499291|
|         61|2499302|
|          5|2499327|
|         48|2499361|
|          3|2499362|
|         59|2499369|
|         27|2499446|
|          1|2499463|
|         46|2499475|
|         57|2499504|
|         56|2499508|
|         42|2499510|
|         33|2499516|
|         24|2499556|
|         31|2499561|
|         23|2499568|
|         38|2499610|
|         53|2499630|
|         35|2499638|
|          8|2499639|
|         18|2499643|
|         11|2499704|
|         15|2499775|
|         44|2499785|
|         28|2499797|
|         50|2499811|
|         19|2499821|
|         

In [19]:
displaypartitions(flight_2021_df)

Partitions: 17


+-----------+------+
|partitionId| count|
+-----------+------+
|         16|188526|
|         12|379865|
|          3|380152|
|          1|381670|
|         10|381716|
|          5|381723|
|          7|381968|
|         14|381978|
|          4|382050|
|          6|382277|
|          0|382585|
|          8|382902|
|          2|384001|
|         11|384797|
|         15|385011|
|          9|385289|
|         13|385361|
+-----------+------+



In [20]:
#re-partition the data
weather_df = weather_df.repartition(27)
displaypartitions(weather_df)

Partitions: 27


+-----------+-------+
|partitionId|  count|
+-----------+-------+
|         12|5771880|
|         14|5771882|
|         13|5771882|
|          8|5771882|
|          7|5771883|
|         11|5771883|
|          9|5771883|
|         19|5771884|
|          4|5771885|
|         15|5771885|
|         10|5771885|
|         20|5771885|
|          6|5771885|
|         17|5771886|
|         25|5771886|
|         18|5771886|
|         16|5771887|
|          3|5771887|
|         26|5771887|
|         21|5771887|
|          0|5771887|
|          5|5771887|
|          1|5771888|
|         24|5771888|
|          2|5771888|
|         22|5771889|
|         23|5771889|
+-----------+-------+



In [22]:
#re-partition the data
flight_2021_df = flight_2021_df.repartition(17)
displaypartitions(flight_2021_df)

Partitions: 17


+-----------+------+
|partitionId| count|
+-----------+------+
|          3|371283|
|          4|371285|
|          5|371285|
|          2|371285|
|          6|371286|
|         12|371286|
|         13|371287|
|          8|371287|
|          7|371287|
|         11|371287|
|         15|371287|
|         10|371287|
|         14|371287|
|          1|371288|
|         16|371288|
|          9|371288|
|          0|371288|
+-----------+------+



### Convert Date Format

In [32]:
new = weather_df.withColumn("date_key", to_date(weather_df['DATE'], "MM/dd/yyyy"))
new.show(2, vertical = True)

-RECORD 0----------------
 ID        | USC00027390 
 DATE      | 4/13/2001   
 TMAX      | 200         
 TMIN      | 17          
 EVAP      | 69          
 PRCP      | 0           
 Latitude  | 32.815      
 Longitude | -109.6808   
 Elevation | 900.4       
 date_key  | 2001-04-13  
-RECORD 1----------------
 ID        | US1TXCRL003 
 DATE      | 6/6/2017    
 TMAX      | null        
 TMIN      | null        
 EVAP      | null        
 PRCP      | 328         
 Latitude  | 31.0916     
 Longitude | -97.89      
 Elevation | 311.2       
 date_key  | 2017-06-06  
only showing top 2 rows



In [33]:
new.printSchema()

root
 |-- ID: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- EVAP: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- date_key: date (nullable = true)



In [28]:
weather_df_2021 = new.filter(new['date_key'] >= '2021-01-01')

In [34]:
weather_df_2021.printSchema()

root
 |-- ID: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- EVAP: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- date_key: date (nullable = true)



In [40]:
weather_df_2021.count()

3406893

In [35]:
displaypartitions(weather_df_2021)

KeyboardInterrupt: 

In [36]:
weather_df_2021 = weather_df_2021.withColumn("latitude_clean", weather_df["Latitude"].cast("double"))
weather_df_2021 = weather_df_2021.withColumn("longitude_clean", weather_df["Longitude"].cast("double"))

In [37]:
weather_df_2021.printSchema()

root
 |-- ID: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- EVAP: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- date_key: date (nullable = true)
 |-- latitude_clean: double (nullable = true)
 |-- longitude_clean: double (nullable = true)



In [38]:
airport_df.printSchema()

root
 |-- iata: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [39]:
airport_df = airport_df.withColumn('latitude_airport', airport_df['latitude'].cast("double"))
airport_df = airport_df.withColumn('longitude_airport', airport_df['longitude'].cast("double"))
airport_df = airport_df.withColumnRenamed("latitude","latitude_weather")\
                       .withColumnRenamed("longitude","longitude_wearher")
airport_df.show(2, vertical = True)

-RECORD 0---------------------------------
 iata              | 00M                  
 name              | Thigpen              
 city              | Bay Springs          
 state             | MS                   
 country           | USA                  
 latitude_weather  | 31.95376472          
 longitude_wearher | -89.23450472         
 latitude_airport  | 31.95376472          
 longitude_airport | -89.23450472         
-RECORD 1---------------------------------
 iata              | 00R                  
 name              | Livingston Municipal 
 city              | Livingston           
 state             | TX                   
 country           | USA                  
 latitude_weather  | 30.68586111          
 longitude_wearher | -95.01792778         
 latitude_airport  | 30.68586111          
 longitude_airport | -95.01792778         
only showing top 2 rows



In [ ]:
# Haversine formula to calculate distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Register the UDF
haversine_udf = udf(haversine, DoubleType())

# Cross join the location and airport DataFrames
df_cross = weather_df_2021.crossJoin(airport_df)

# Calculate the Haversine distance for each location-airport pair
df_with_distance = df_cross.withColumn(
    "distance_km",
    haversine_udf(
        col("latitude_clean"), col("longitude_clean"),
        col("latitude_airport"), col("longitude_airport")
    )
)

# Define a window partitioned by location and ordered by distance
windowSpec = Window.partitionBy("latitude_clean", "longitude_clean").orderBy("distance_km")

# Select the nearest airport by ordering by distance and choosing the first entry
df_nearest_airport = df_with_distance.withColumn("rank", row_number().over(windowSpec)) \
                                     .filter(col("rank") == 1) \
                                     .select('ID','DATE','TMAX','TMIN','EVAP','PRCP','Latitude',
                                             'Longitude','Elevation','date_key','latitude_clean',
                                             'longitude_clean', 'iata','name','city','state',"distance_km")

# Show the result
df_nearest_airport.show()

24/11/04 03:21:59 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1730686805282_0002_01_000019 on host: hub-hub-msca-bdp-dphub-student-andrewrafael-sw-d593.c.msca-bdp-student-ap.internal. Exit status: 143. Diagnostics: [2024-11-04 03:21:59.150]Container killed on request. Exit code is 143
[2024-11-04 03:21:59.162]Container exited with a non-zero exit code 143. 
[2024-11-04 03:21:59.162]Killed by external signal
.
24/11/04 03:21:59 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1730686805282_0002_01_000020 on host: hub-hub-msca-bdp-dphub-student-andrewrafael-sw-d593.c.msca-bdp-student-ap.internal. Exit status: 143. Diagnostics: [2024-11-04 03:21:59.149]Container killed on request. Exit code is 143
[2024-11-04 03:21:59.149]Container exited with a non-zero exit code 143. 
[2024-11-04 03:21:59.161]Killed by external signal
.
24/11/04 03:21:59 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedul

In [38]:
weather_df_2021.columns

['ID',
 'DATE',
 'TMAX',
 'TMIN',
 'EVAP',
 'PRCP',
 'Latitude',
 'Longitude',
 'Elevation',
 'date_key',
 'latitude_clean',
 'longitude_clean']